In [5]:
# === Append mínimo MAFI (local, sin DVC) ===
from pathlib import Path
import pandas as pd
import re

# ----------------- CONFIG -----------------
# Ajusta estas rutas si tus nombres difieren
BASE = Path.cwd()
P1 = BASE / "data_raw" / "mafi" / "mafi_2021_2024_01.xlsx"  # o "data/MAFI 2021-2024 1.xlsx"
P2 = BASE / "data_raw" / "mafi" / "mafi_2025_01.xlsx"       # o "data/MAFI 2025 (1).xlsx"

OUTDIR = BASE / "data_curated"
OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_CSV  = OUTDIR / "mafi_common_append.csv"
OUT_PARQ = OUTDIR / "mafi_common_append.parquet"  # requiere pyarrow o fastparquet (opcional)

# Sinónimos mínimos (izq = como podría venir; der = nombre estándar)
SYN = {
    "idbanner": "id_estudiante",
    "tipoestudiante": "tipo_estudiante",
    "codprograma": "cod_programa",
    "plan": "plan_estudio",
}

# Columnas preferidas primero (si existen)
PREFER = [
    "id_estudiante","cohorte","periodo","facultad","programa","cod_programa",
    "nivel","tipo_estudiante","sede","jornada","estrato","ingreso",
    "ruta_academica","plan_estudio","estado","estado_usuario","genero",
    "ciudad","codciudad","nacionalidad","fecha_nacimiento"
]

# ----------------- HELPERS -----------------
def norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    """normaliza encabezados -> snake_case simple"""
    cols = []
    for c in df.columns:
        c2 = str(c).strip().lower()
        c2 = re.sub(r"[^\w\s]", " ", c2)
        c2 = re.sub(r"\s+", "_", c2).strip("_")
        cols.append(c2)
    out = df.copy()
    out.columns = cols
    return out

def read_first_sheet(path: Path) -> pd.DataFrame:
    # dtype=str evita inferencia pesada y mismatches; luego casteamos lo necesario
    return pd.read_excel(path, engine="openpyxl", dtype=str)

def apply_synonyms(df: pd.DataFrame, syn: dict) -> pd.DataFrame:
    # renombra solo si existe la columna original
    usable = {k: v for k, v in syn.items() if k in df.columns}
    return df.rename(columns=usable)

# ----------------- PIPELINE -----------------
# 1) Lectura rápida (local)
df1 = read_first_sheet(P1)
df2 = read_first_sheet(P2)

# 2) Normaliza columnas y aplica sinónimos (una sola vez)
df1 = apply_synonyms(norm_cols(df1), SYN)
df2 = apply_synonyms(norm_cols(df2), SYN)

# 3) Intersección de columnas y orden final (preferidas primero)
common = sorted(set(df1.columns) & set(df2.columns))
final_cols = [c for c in PREFER if c in common] + [c for c in common if c not in PREFER]

# 4) Append
mafi = pd.concat([df1[final_cols], df2[final_cols]], ignore_index=True)

# 5) Casts ligeros (solo si existen)
for c in ("periodo","cohorte","estrato","codciudad"):
    if c in mafi.columns:
        mafi[c] = pd.to_numeric(mafi[c], errors="coerce").astype("Int64")
if "ingreso" in mafi.columns:
    mafi["ingreso"] = pd.to_numeric(mafi["ingreso"], errors="coerce")

# 6) Resumen y guardado
print(f"Filas totales: {len(mafi):,} | Columnas comunes: {len(final_cols)}")
print("Primeras columnas:", final_cols[:12])

mafi.to_csv(OUT_CSV, index=False)
print("CSV guardado en:", OUT_CSV)

# Parquet opcional (comenta si no lo necesitas o si no tienes pyarrow/fastparquet)
try:
    mafi.to_parquet(OUT_PARQ, index=False)
    print("Parquet guardado en:", OUT_PARQ)
except Exception as e:
    print("Parquet omitido (instala 'pyarrow' o 'fastparquet' si lo necesitas). Detalle:", e)


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Juanr\\OneDrive\\Escritorio\\Taller3-MLFlow\\data_raw\\mafi\\mafi_2021_2024_01.xlsx'